## Dataframe of sentences

In [2]:
from twitterscraper import query_tweets
import codecs, json
import re
import operator
import copy
import nltk
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, SpaceTokenizer, PunktSentenceTokenizer
from collections import Counter
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import copy

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13'}


In [4]:
with codecs.open('collection.json', 'r', 'utf-8') as f:
    tw = json.load(f, encoding = 'utf-8')

In [3]:
tweet_table = pd.DataFrame(tw)

In [4]:
expand_table = copy.deepcopy(tweet_table)

In [5]:
expand_table["text"] = expand_table["text"].str.split("[!?.]")

In [6]:
expand_table = expand_table.explode("text")

In [7]:
print(len(expand_table))
print(len(tweet_table))

345768
122564


In [74]:
#expand_table["text_split"] = expand_table["text"].str.split(" ")

In [75]:
notemo = expand_table[~expand_table["text"].str.contains("please")]
notemo = notemo[~notemo["text"].str.contains("Please")]
notemo = notemo[~notemo["text"].str.contains("PLEASE")]
print(len(notemo))

338088


In [76]:
notemo = notemo[~notemo["text"].str.contains("help")]
notemo = notemo[~notemo["text"].str.contains("Help")]
notemo = notemo[~notemo["text"].str.contains("HELP")]
print(len(notemo))

323795


In [77]:
notemo = notemo[~notemo["text"].str.contains("pray")]
notemo = notemo[~notemo["text"].str.contains("PRAY")]
notemo = notemo[~notemo["text"].str.contains("Pray")]
print(len(notemo))

319640


In [78]:
notemo = notemo[~notemo["text"].str.contains("climate change")]
notemo = notemo[~notemo["text"].str.contains("Climate change")]
notemo = notemo[~notemo["text"].str.contains("CLIMATE CHANGE")]
notemo = notemo[~notemo["text"].str.contains("Climate Change")]
print(len(notemo))

315643


In [79]:
notemo = notemo[~notemo["text"].str.contains("thank you")]
notemo = notemo[~notemo["text"].str.contains("THANK YOU")]
notemo = notemo[~notemo["text"].str.contains("Thank you")]
notemo = notemo[~notemo["text"].str.contains("Thank You")]
notemo = notemo[~notemo["text"].str.contains("Thank")]
notemo = notemo[~notemo["text"].str.contains("thank")]
notemo = notemo[~notemo["text"].str.contains("THANK")]
print(len(notemo))

311064


In [80]:
notemo = notemo[~notemo["text"].str.contains("donate")]
notemo = notemo[~notemo["text"].str.contains("DONATE")]
notemo = notemo[~notemo["text"].str.contains("Donate")]
print(len(notemo))

306716


In [81]:
notemo = notemo[~notemo["text"].str.contains("OnFire")]
print(len(notemo))

304989


In [82]:
notemo = notemo[~notemo["text"].str.contains("Bushfires Burning")]
notemo = notemo[~notemo["text"].str.contains("Emergency Fires")]
notemo = notemo[~notemo["text"].str.contains("Burning Emergency")]
print(len(notemo))

304540


In [83]:
notemo = notemo[~notemo["text"].str.contains("link")]
notemo = notemo[~notemo["text"].str.contains("Link")]
notemo = notemo[~notemo["text"].str.contains("LINK")]
print(len(notemo))

303560


In [85]:
notemo = notemo[~notemo["text"].str.contains("nBushfire")]
notemo = notemo[~notemo["text"].str.contains("isBurning")]
notemo = notemo[~notemo["text"].str.contains("nWildFires")]
notemo = notemo[~notemo["text"].str.contains("IsOnFire")]
notemo = notemo[~notemo["text"].str.contains("nBushfiresDisaster")]
print(len(notemo))

301970


In [86]:
notemo = notemo[~notemo["text"].str.contains("Click here")]
notemo = notemo[~notemo["text"].str.contains("click here")]
notemo = notemo[~notemo["text"].str.contains("CLICK HERE")]
print(len(notemo))

301919


In [87]:
punct_list = list(punctuation)
punct_list.append('``')
noise = stopwords.words('english') + punct_list

In [88]:
for j in ['…', '\'', '’', '', '\'s', '...', '/…', 'n\'t', '\'\'', 'us', '“', '”', '\'re', 'one', '\'m', 'australia', 'this']:
    noise.append(j)

for i in range(0, 2050):
    noise.append(str(i))

In [89]:
lemmas = WordNetLemmatizer()

In [90]:
notemo['text'].iloc[0]

"'Act immediately to survive': Fires rip to Australia's southeast"

In [91]:
notemo['text'].iloc[1]

' Find out more here:     '

In [92]:
preprop_text = []
for i in range(len(notemo['text'])):
    the_text = notemo['text'].iloc[i]
    tok_list = word_tokenize(the_text)   
    sent = ''
    for word in tok_list:
        if word.lower() not in noise:
            my_word = str(word.lower())
            my_word_num = re.sub(r" .*?[0-9]*.*? ", " ", my_word)
            my_word_num = re.sub(r"[.\'/]", "", my_word_num)
            sent += lemmas.lemmatize(my_word_num)
            sent += ' '
    preprop_text.append(sent)

In [93]:
notemo['preprop'] = preprop_text

In [94]:
notemo_2020 = notemo.query('timestamp >= "2020-01-01T00:00:00" and timestamp <= "2020-03-01T00:00:00"')

In [95]:
notemo_2020_sorted = notemo_2020.sort_values(by = 'timestamp')

In [96]:
time_cluster = []
for i in range(len(notemo_2020_sorted)):
    time_cluster.append(i // (len(notemo_2020_sorted) / 14) + 1)
    
len(time_cluster)
#notemo_2020_sorted['time_cluster'] = time_cluster

270659

In [97]:
notemo_2020_sorted['time_cluster'] = time_cluster

In [98]:
len(notemo_2020_sorted) / 14

19332.785714285714

In [99]:
tfidf_vectorizer = TfidfVectorizer()

In [2]:
#np.array(tweet_2020_n['preprop'])

In [163]:
def summarization(n):
    tweet_2020_n = notemo_2020_sorted.query('time_cluster == %d' % n)
    tfidf_n = tfidf_vectorizer.fit_transform(np.array(tweet_2020_n['preprop']))
    kmeans = KMeans(n_clusters=3).fit(tfidf_n)

    print("End Kmeans")
    cluster_of_3 = []
    for i in range(len(tweet_2020_n)):
        cluster_of_3.append(kmeans.predict(tfidf_n.getrow(i))[0])

    tweet_2020_n['cluster_3'] = np.array(cluster_of_3)

    distances = []
    for i in range(len(tweet_2020_n)):
        center = np.array(tweet_2020_n['cluster_3'])[i]
        vec_sub = kmeans.cluster_centers_[center] - tfidf_n.todense()[i]
        if i % 500 == 0:
            print(i)
        vec_np = np.array(vec_sub)
        vec_np_2 = vec_np ** 2
        dist = vec_np_2.sum()
        distances.append(dist)
    
    tweet_2020_n['distances'] = distances
    print("End distances")
    
    length = []
    for i in range(len(tweet_2020_n['preprop'])):
        ll = len(np.array(tweet_2020_n['preprop'])[i].split())
        length.append(ll)

    tweet_2020_n['preprop_len'] = length

    return tweet_2020_n

In [164]:
tweet_2020_5 = summarization(5)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [165]:
list(tweet_2020_5.sort_values(by = 'distances').query('cluster_3 == 1 and preprop != "" and preprop_len > 3 ')['text'])

[' Australian fires: Why do people start fires during fires',
 ' Australian fires: Why do people start fires during fires',
 ' Australian fires: Why do people start fires during fires',
 '  Australian fires: Why do people start fires during fires',
 '  Australian fires: Why do people start fires during fires',
 'Australian fires …     Burning Fires isBurning',
 ' - The Perfect Firestorm -  By Design…Fire OnFire Fires Fires Burns Burning        ',
 'The fires are bigger than California fires in 2018',
 'Australia fire animals are burning video footage fire   ',
 'Australia is Literally on fire, and its the one of the worst fires in recent years worldwide, Bigger than the Amazon fires',
 '“Australian fires have burned more than twice the area than was consumed, combined, by California’s 2018 fires and the Amazon’s 2019 fires',
 ' its beauty is slowly being destroyed by fire Fires    Burns Bushfires',
 ' its beauty is slowly being destroyed by fire Fires    Burns Bushfires',
 'It is sad s

In [166]:
tweet_2020_1 = summarization(1)
tweet_2020_1.to_csv('notemo_2020_1.csv')

tweet_2020_2 = summarization(2)
tweet_2020_2.to_csv('notemo_2020_2.csv')

tweet_2020_3 = summarization(3)
tweet_2020_3.to_csv('notemo_2020_3.csv')

tweet_2020_4 = summarization(4)
tweet_2020_4.to_csv('notemo_2020_4.csv')

tweet_2020_5 = summarization(5)
tweet_2020_5.to_csv('notemo_2020_5.csv')

tweet_2020_6 = summarization(6)
tweet_2020_6.to_csv('notemo_2020_6.csv')

tweet_2020_7 = summarization(7)
tweet_2020_7.to_csv('notemo_2020_7.csv')

tweet_2020_8 = summarization(8)
tweet_2020_8.to_csv('notemo_2020_8.csv')

tweet_2020_9 = summarization(9)
tweet_2020_9.to_csv('notemo_2020_9.csv')

tweet_2020_10 = summarization(10)
tweet_2020_10.to_csv('notemo_2020_10.csv')

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


...

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [170]:
tweet_2020_11 = summarization(11)
tweet_2020_11.to_csv('notemo_2020_11.csv')

tweet_2020_12 = summarization(12)
tweet_2020_12.to_csv('notemo_2020_12.csv')

tweet_2020_13 = summarization(13)
tweet_2020_13.to_csv('notemo_2020_13.csv')

tweet_2020_14 = summarization(14)
tweet_2020_14.to_csv('notemo_2020_14.csv')

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


...

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Naive Bayes NO but removed spam

In [100]:
length = []
for i in range(len(notemo_2020_sorted['preprop'])):
    ll = len(np.array(notemo_2020_sorted['preprop'])[i].split())
    length.append(ll)

In [101]:
notemo_2020_sorted['preprop_len'] = length

In [102]:
print(len(notemo_2020_sorted))
print(len(notemo_2020_sorted.query('text != ""')))
print(len(notemo_2020_sorted.query('preprop != ""')))
print(len(notemo_2020_sorted.query('preprop_len > 3')))

270659
226057
172460
102070


In [103]:
notemo_adeq = notemo_2020_sorted.query('preprop_len > 3')

In [109]:
len(notemo_adeq.drop_duplicates(subset=['text'], keep=False))

94829

In [111]:
notemo_adeq = notemo_adeq.drop_duplicates(subset=['text'], keep=False)

In [122]:
#notemo_adeq

In [120]:
time_cluster = []
for i in range(len(notemo_adeq)):
    time_cluster.append(i // (len(notemo_adeq) / 10) + 1)
    
#len(time_cluster)
notemo_adeq['time_clust'] = time_cluster
#notemo_adeq.drop(['time_cluster'])

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [124]:
tfidf_vectorizer = TfidfVectorizer()

from tqdm import tqdm

In [125]:
def summarization_adeq(n):
    tweet_2020_n = notemo_adeq.query('time_cluster == %d' % n)
    tfidf_n = tfidf_vectorizer.fit_transform(np.array(tweet_2020_n['preprop']))
    kmeans = KMeans(n_clusters=3).fit(tfidf_n)

    print("End Kmeans")
    cluster_of_3 = []
    for i in range(len(tweet_2020_n)):
        cluster_of_3.append(kmeans.predict(tfidf_n.getrow(i))[0])

    tweet_2020_n['cluster_3'] = np.array(cluster_of_3)

    distances = []
    for i in tqdm(range(len(tweet_2020_n))):
        center = np.array(tweet_2020_n['cluster_3'])[i]
        vec_sub = kmeans.cluster_centers_[center] - tfidf_n.todense()[i]
        #if i % 500 == 0:
         #   print(i)
        vec_np = np.array(vec_sub)
        vec_np_2 = vec_np ** 2
        dist = vec_np_2.sum()
        distances.append(dist)
    
    tweet_2020_n['distances'] = distances
    print("End distances")
    
    length = []
    for i in range(len(tweet_2020_n['preprop'])):
        ll = len(np.array(tweet_2020_n['preprop'])[i].split())
        length.append(ll)

    tweet_2020_n['preprop_len'] = length

    return tweet_2020_n

In [126]:
sent_2020_1 = summarization_adeq(1)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 9483/9483 [15:00<00:00, 10.53it/s]
/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [130]:
sent_2020_1.to_csv('sent_2020_1.csv')

In [131]:
sent_2020_2 = summarization_adeq(2)
sent_2020_2.to_csv('sent_2020_2.csv')

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 9483/9483 [14:35<00:00, 10.83it/s]
/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [132]:
sent_2020_4 = summarization_adeq(4)
sent_2020_4.to_csv('sent_2020_4.csv')

sent_2020_3 = summarization_adeq(3)
sent_2020_3.to_csv('sent_2020_3.csv')

sent_2020_5 = summarization_adeq(5)
sent_2020_5.to_csv('sent_2020_5.csv')

sent_2020_6 = summarization_adeq(6)
sent_2020_6.to_csv('sent_2020_6.csv')

sent_2020_7 = summarization_adeq(7)
sent_2020_7.to_csv('sent_2020_7.csv')

sent_2020_8 = summarization_adeq(8)
sent_2020_8.to_csv('sent_2020_8.csv')

sent_2020_9 = summarization_adeq(9)
sent_2020_9.to_csv('sent_2020_9.csv')

sent_2020_10 = summarization_adeq(10)
sent_2020_10.to_csv('sent_2020_10.csv')

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 9483/9483 [15:13<00:00, 10.38it/s]
/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End Kmeans


...

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 9482/9482 [15:30<00:00, 10.19it/s]
/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Probability method

In [3]:
sent_2020_1 = pd.read_csv('sent_2020_1.csv')
sent_2020_2 = pd.read_csv('sent_2020_2.csv')
sent_2020_3 = pd.read_csv('sent_2020_3.csv')
sent_2020_4 = pd.read_csv('sent_2020_4.csv')
sent_2020_5 = pd.read_csv('sent_2020_5.csv')
sent_2020_6 = pd.read_csv('sent_2020_6.csv')
sent_2020_7 = pd.read_csv('sent_2020_7.csv')
sent_2020_8 = pd.read_csv('sent_2020_8.csv')
sent_2020_9 = pd.read_csv('sent_2020_9.csv')
sent_2020_10 = pd.read_csv('sent_2020_10.csv')

In [5]:
def freq_sum(table, n):
    sent_2020 = table
    sent_2020 = sent_2020.query('cluster_3 == %d' % n)
    _ = list(sent_2020['preprop'])

    all_freq = Counter()
    all_nums = 0

    for i in range(len(_)):
        freq_dict = Counter(_[i].split())
        all_freq += freq_dict
        all_nums += len(freq_dict)

    p_ = []
    for i in range(len(_)):
        probab = 0
        sp = _[i].split()
        for j in range(len(sp)):
            probab = np.log(all_freq[sp[j]] / len(sp))

        p_.append(probab)

    sent_2020['probab_sent'] = p_
    return sent_2020

In [6]:
print('\n', 'sent_2020_1_0')
sent_2020_1_0 = freq_sum(sent_2020_1, 0)
for i in range(6):
    print(list(sent_2020_1_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_1_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_1_1')
sent_2020_1_1 = freq_sum(sent_2020_1, 1)
for i in range(6):
    print(list(sent_2020_1_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_1_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
print('\n', 'sent_2020_1_2')
sent_2020_1_2 = freq_sum(sent_2020_1, 2)
for i in range(6):
    print(list(sent_2020_1_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_1_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_2_0')
sent_2020_2_0 = freq_sum(sent_2020_2, 0)
for i in range(6):
    print(list(sent_2020_2_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_2_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_2_1')
sent_2020_2_1 = freq_sum(sent_2020_2, 1)
for i in range(6):
    print(list(sent_2020_2_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_2_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_2_2')
sent_2020_2_2 = freq_sum(sent_2020_2, 2)
for i in range(6):
    print(list(sent_2020_2_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_2_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_3_0')
sent_2020_3_0 = freq_sum(sent_2020_3, 0)
for i in range(6):
    print(list(sent_2020_3_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_3_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_3_1')
sent_2020_3_1 = freq_sum(sent_2020_3, 1)
for i in range(6):
    print(list(sent_2020_3_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_3_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_3_2')
sent_2020_3_2 = freq_sum(sent_2020_3, 2)
for i in range(6):
    print(list(sent_2020_3_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_3_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_4_0')
sent_2020_4_0 = freq_sum(sent_2020_4, 0)
for i in range(6):
    print(list(sent_2020_4_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_4_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_4_1')
sent_2020_4_1 = freq_sum(sent_2020_4, 1)
for i in range(6):
    print(list(sent_2020_4_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_4_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_4_2')
sent_2020_4_2 = freq_sum(sent_2020_4, 2)
for i in range(6):
    print(list(sent_2020_4_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_4_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_5_0')
sent_2020_5_0 = freq_sum(sent_2020_5, 0)
for i in range(6):
    print(list(sent_2020_5_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_5_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_5_1')
sent_2020_5_1 = freq_sum(sent_2020_5, 1)
for i in range(6):
    print(list(sent_2020_5_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_5_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_5_2')
sent_2020_5_2 = freq_sum(sent_2020_5, 2)
for i in range(6):
    print(list(sent_2020_5_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_5_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])



 sent_2020_1_0


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


My Heart weeps for the Australian people
2020-01-04T04:26:04
 Just fucking stop touching people
2020-01-04T02:43:42
 Scomo won’t even listen to the people
2020-01-03T01:33:33
Good sense from great people
2020-01-03T02:43:20
 just called those poor terrified people at  
2020-01-03T23:13:19
  They have been negligent in their duties to the Australian people
2020-01-03T21:47:02

 sent_2020_1_1
Over  200 arsonists caught lighting fires in Australia
2020-01-04T01:07:39
 Two people have died in the  fires
2020-01-04T00:29:32
 Many parts of Australia are currently on Fire
2020-01-03T06:30:40
24 Million acres2019-20 Australian Fires: 12
2020-01-03T06:34:08
Has there been a  established for the people affected by the fires in Australia
2020-01-03T16:24:32
Humanity , Which not burned in Australian Fire 
2020-01-03T16:23:21

 sent_2020_1_2
My heart’s being ripped apart for Australia’s animals
2020-01-03T12:51:56
 I know you’re a brand who loves animals
2020-01-03T22:57:30
 Nature vs Humans, Anima

In [7]:
print('\n', 'sent_2020_6_0')
sent_2020_6_0 = freq_sum(sent_2020_6, 0)
for i in range(6):
    print(list(sent_2020_6_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_6_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_6_1')
sent_2020_6_1 = freq_sum(sent_2020_6, 1)
for i in range(6):
    print(list(sent_2020_6_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_6_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_6_2')
sent_2020_6_2 = freq_sum(sent_2020_6, 2)
for i in range(6):
    print(list(sent_2020_6_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_6_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_7_0')
sent_2020_7_0 = freq_sum(sent_2020_7, 0)
for i in range(6):
    print(list(sent_2020_7_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_7_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_7_1')
sent_2020_7_1 = freq_sum(sent_2020_7, 1)
for i in range(6):
    print(list(sent_2020_7_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_7_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_7_2')
sent_2020_7_2 = freq_sum(sent_2020_7, 2)
for i in range(6):
    print(list(sent_2020_7_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_7_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_8_0')
sent_2020_8_0 = freq_sum(sent_2020_8, 0)
for i in range(6):
    print(list(sent_2020_8_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_8_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_8_1')
sent_2020_8_1 = freq_sum(sent_2020_8, 1)
for i in range(6):
    print(list(sent_2020_8_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_8_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_8_2')
sent_2020_8_2 = freq_sum(sent_2020_8, 2)
for i in range(6):
    print(list(sent_2020_8_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_8_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_9_0')
sent_2020_9_0 = freq_sum(sent_2020_9, 0)
for i in range(6):
    print(list(sent_2020_9_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_9_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_9_1')
sent_2020_9_1 = freq_sum(sent_2020_9, 1)
for i in range(6):
    print(list(sent_2020_9_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_9_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_9_2')
sent_2020_9_2 = freq_sum(sent_2020_9, 2)
for i in range(6):
    print(list(sent_2020_9_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_9_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_10_0')
sent_2020_10_0 = freq_sum(sent_2020_10, 0)
for i in range(6):
    print(list(sent_2020_10_0.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_10_0.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])

print('\n', 'sent_2020_10_1')
sent_2020_10_1 = freq_sum(sent_2020_10, 1)
for i in range(6):
    print(list(sent_2020_10_1.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_10_1.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])
    
print('\n', 'sent_2020_10_2')
sent_2020_10_2 = freq_sum(sent_2020_10, 2)
for i in range(6):
    print(list(sent_2020_10_2.sort_values(by = 'probab_sent', ascending = False)['text'])[i])
    print(list(sent_2020_10_2.sort_values(by = 'probab_sent', ascending = False)['timestamp'])[i])



 sent_2020_6_0


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The world is speaking out for Australian people
2020-01-10T08:05:24
My heart is filled with sorrow for the people of Australia
2020-01-09T17:42:34
 Our hearts go out to the brave people of Australia
2020-01-09T14:45:29
 He's destroyed what we had, so now he has to suffer the wrath of the people
2020-01-10T06:32:14
this is a time for paying attention people
2020-01-09T22:47:28
 It’s scary how little shame these people have
2020-01-09T18:33:08

 sent_2020_6_1
 We are not in the immediate area of bush fires
2020-01-09T13:56:27
"Arson is not the reason for the catastrophic fires in Australia
2020-01-10T13:09:45
 Envision rains putting out all fires
2020-01-09T11:11:51
 hope it continues over at least some fires   
2020-01-10T00:31:59
 Now, 12 yrs later, worse fires
2020-01-10T11:45:05
This is not to going to fix your problem  Fires …
2020-01-10T01:05:43

 sent_2020_6_2
Human lives are as precious as Animals 
2020-01-09T17:19:54
We need a second mobilisation for the animals
2020-01-09T23:07